In [42]:
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np

## Use beatifulsoup to load the data
Source: https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969

In [43]:
source = urllib.request.urlopen('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969').read()
soup = bs.BeautifulSoup(source,'lxml')
df = pd.DataFrame(columns={'PostalCode','Borough', 'Neighborhood'})

table_contents=[]
table=soup.find('table', attrs={'class':'wikitable sortable'})

# Load the data in a dataframe

In [44]:
for row in table.find_all('tr'):
    code=''
    borough=''
    neighbourhood=''
    for col in row.find_all('td'):
        if code=='':
            code=col.text          
        elif borough=='':
            borough=col.text         
        elif neighbourhood=='':    
            neighbourhood=col.text            
            if neighbourhood== 'Not assigned':
                neighbourhood=borough
    if borough != 'Not assigned\n' and code != '':
        df2=pd.DataFrame({'Postal Code': [code.replace("\n","")],'Borough': [borough.replace("\n","")],'Neighborhood': [neighbourhood.replace("\n","")]})
        df=df.append(df2, ignore_index = True, sort=False)
        df =df[['Postal Code','Borough', 'Neighborhood']]

            
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Get the dimensions of the dataframe

In [45]:
df.shape

(103, 3)

# Use the Geocoder to get the latitude and Longitude of each Postal Code

In [1]:
%pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [12]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
for i in range(len(df)):
    print(df.loc[i, 'PostalCode'])
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(df.loc[i, 'PostalCode']))
        lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude)
print(longitude)

M3A


KeyboardInterrupt: 

# Use the provided CSV file instead to import the latitude and Longitude for each Postal Code
Alternative solution, could not make the geocoder worked

In [41]:
data = pd.read_csv("Geospatial_Coordinates.csv")
result = pd.merge(df,
                 data[['Postal Code', 'Latitude', 'Longitude']],
                 on='Postal Code')
result.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
